# Association pattern mining

Apriori algorithm and an algorithm for generating association rules.

i will apply the implemented algorithms to Titanic dataset to identify which groups of passengers were likely to survive/not survive in the catastrophe.

Each record has 4 features:
- Class (0 = crew, 1 = first, 2 = second, 3 = third)
- Age   (1 = adult, 0 = child)
- Sex   (1 = male, 0 = female)
- Survived (1 = yes, 0 = no)

For the Frequent Pattern Mining Model and Apriori algorithm we need to convert our dataset to a binary dataset using [one-hot encoding] 

Using onew-hot encoding, the original four features of our dataset will be converted into the following 10 binary features:
- Crew
- 1st class
- 2nd class
- 3rd class
- Child
- Adult
- Female
- Male
- Survived
- Not survived

In [1]:
import numpy as np



Read Titanic dataset from the input file and transform it using one-hot encoding

In [2]:
# Read and convert dataset using one-hot encoding
def load_data_one_hot(fname):
    itemNames = ['Crew', '1st class', '2nd class', '3rd class', 'Child', 'Adult', 'Female', 'Male', 'Survived', 'Not survived']
    dataset = []
    
    with open(fname) as F:
        next(F) # skip the first line with feature names
        for line in F:
            p = line.strip().split('        ')
            X = np.array(p, int)
            newX = [0]*10
            if X[0] == 0: newX[0] = 1 # Crew
            if X[0] == 1: newX[1] = 1 # First class
            if X[0] == 2: newX[2] = 1 # Second class
            if X[0] == 3: newX[3] = 1 # Thirds class
            if X[1] == 0: newX[4] = 1 # Child
            if X[1] == 1: newX[5] = 1 # Adult
            if X[2] == 0: newX[6] = 1 # Female
            if X[2] == 1: newX[7] = 1 # Male   
            if X[3] == 1: newX[8] = 1 # Survived
            if X[3] == 0: newX[9] = 1 # Not survived
            dataset.append(newX)    
    
    return np.array(itemNames), np.array(dataset)

In [3]:
itemNames, dataset = load_data_one_hot('titanic.dat.txt')
print(itemNames)
print(dataset)

['Crew' '1st class' '2nd class' '3rd class' 'Child' 'Adult' 'Female'
 'Male' 'Survived' 'Not survived']
[[0 1 0 ... 1 1 0]
 [0 1 0 ... 1 1 0]
 [0 1 0 ... 1 1 0]
 ...
 [1 0 0 ... 0 0 1]
 [1 0 0 ... 0 0 1]
 [1 0 0 ... 0 0 1]]



1. Implement Apriori algorithm

In [4]:
# Computes support of a given itemset
def sup(dataset, itemset):
    # compute the number of items in the itemset
    numItems = len(itemset)
    # for every instance consider only items from itemset
    resDataset = dataset[:,itemset]
    
    return np.sum(np.sum(resDataset, axis=1) == numItems)/len(dataset)

In [5]:
# Given the frequent (k-1)-itemsets, generate the candidate k-itemsets
def generateCandidates(F, k, d):
    # Create an empty list of candidate k-itemsets
    C = []

    ### JOIN PHASE
    # for each frequent itemset
    for itemset in F:
        # and for every i in the range from the value of the last element in the itemset + 1 to d (the number of elements in the universe)
        for i in range(itemset[-1]+1,d+1):
            # construct itemset1 by replacing the last element in the itemset with i
            itemset1 = itemset[:-1] + [i]
            # if itemset1 is frequent k-itemset
            if itemset1 in F:
                # add (itemset union {i}) to the collection of candidate k-itemsets
                C.append(itemset+[i])

    ### PRUNE PHASE
    # for each candidate k-itemset
    for itemset in C:
        # check if it satisfies the Downward Closure Property
        for i in itemset:
            tempItemset = itemset.copy()
            tempItemset.remove(i)
            if tempItemset not in F:
                C.remove(itemset)
                break
            
    return C

In [6]:
# Generate all frequent itemsets for a given frequency threshold
def apriori(dataset, freqThreshold):
    # compute the number of instances in the dataset
    N = len(dataset)
    # compute the number items in the universe
    d = len(dataset[0])

    # Frequent itemsets will be stored in a dictionary. Key -- size of itemsets, Value -- all frequent itemsets of that size
    F = {i+1 : [] for i in range(d)}

    # Compute frequent 1-itemsets
    for i in range(d):
        if sup(dataset, [i]) >= freqThreshold:
            F[1].append([i])

    # For every itemsets size from 2 to d
    for k in range(2,d+1):
        # If there are no frequent (k-1)-itemsets, then STOP
        if len(F[k-1]) == 0:
            break
        # Otherwise generate k-itemsets candidates
        Ck = generateCandidates(F[k-1], k, d)
        # and for every candidate k-itemset 
        for itemset in Ck:
            # if it has sufficiently large support
            if sup(dataset, itemset) >= freqThreshold:
                # add it to the collections of the frequent k-itemsets
                F[k].append(itemset)

    return F

2. Generate and print all frequent itemsets for the frequency threshold 0.005

In [7]:
F = apriori(dataset, 0.005)
for k,val in F.items():
    if len(val) > 0:
        print('Frequent %d-itemset:' % (k), val)

Frequent 1-itemset: [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]
Frequent 2-itemset: [[0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [1, 5], [1, 6], [1, 7], [1, 8], [1, 9], [2, 4], [2, 5], [2, 6], [2, 7], [2, 8], [2, 9], [3, 4], [3, 5], [3, 6], [3, 7], [3, 8], [3, 9], [4, 6], [4, 7], [4, 8], [4, 9], [5, 6], [5, 7], [5, 8], [5, 9], [6, 8], [6, 9], [7, 8], [7, 9]]
Frequent 3-itemset: [[0, 5, 6], [0, 5, 7], [0, 5, 8], [0, 5, 9], [0, 6, 8], [0, 7, 8], [0, 7, 9], [1, 5, 6], [1, 5, 7], [1, 5, 8], [1, 5, 9], [1, 6, 8], [1, 7, 8], [1, 7, 9], [2, 4, 6], [2, 4, 7], [2, 4, 8], [2, 5, 6], [2, 5, 7], [2, 5, 8], [2, 5, 9], [2, 6, 8], [2, 6, 9], [2, 7, 8], [2, 7, 9], [3, 4, 6], [3, 4, 7], [3, 4, 8], [3, 4, 9], [3, 5, 6], [3, 5, 7], [3, 5, 8], [3, 5, 9], [3, 6, 8], [3, 6, 9], [3, 7, 8], [3, 7, 9], [4, 6, 8], [4, 6, 9], [4, 7, 8], [4, 7, 9], [5, 6, 8], [5, 6, 9], [5, 7, 8], [5, 7, 9]]
Frequent 4-itemset: [[0, 5, 6, 8], [0, 5, 7, 8], [0, 5, 7, 9], [1, 5, 6, 8], [1, 5, 7, 8], [1, 5, 7, 9], [2, 4, 6, 8], [


1. Implement an algorithm that takes frequent itemsets as an input and generate from these itemsets all association rules at a given confidence threshold

In [8]:
# Generate all subsets of a given set s, except the empty set and s itself (the function is implemented as generator, which means that we can use it as iterator)
def powerset(s):
    x = len(s)
    masks = [1 << i for i in range(x)]
    for i in range(1, (1 << x)-1):
        yield [ss for mask, ss in zip(masks, s) if i & mask]

In [9]:
# Given frequent itemsets, generate the association rules at the given confidence threshold
def generateRules(dataset, freqItemsets, confThreshold):
    associationRules = []

    # for every frequent itemset
    for itemset in freqItemsets:
        # of size at least 2
        if len(itemset) >=2:
            # and for every subset X of itemset
            for X in powerset(itemset):
                # if the confidence of X => Y (where Y is the complement of X) is at least the confidence threshold
                support = sup(dataset, itemset)
                conf = support/sup(dataset, X)
                if conf >= confThreshold:
                    # then generate association rule X => Y
                    Y = [e for e in itemset if e not in X]
                    associationRules.append((X,Y,support,conf))
    return associationRules
            

2. Generate and print all association rules at the frequency threshold 0.005 and the confidence threshold 0.8. For every rule, output its support and confidence.

In [10]:
# print association rules
def printAssociationRules(associationRules, itemNames):
    i = 1
    print("Support \t\t Confidence \t\t Association rule")
    for (X,Y,support,conf) in associationRules:
        #print("%d. { %s } => { %s } \t\t\t %.4f \t\t\t %.4f" % i, ','.join(itemNames[X]), ','.join(itemNames[Y]), support, conf)
        print("%d. %.4f \t\t %.4f \t\t { %s } => { %s } \t\t " % (i, support, conf, ', '.join(itemNames[X]), ', '.join(itemNames[Y])))
        #print(i, '{',itemNames[X],'} => {', itemNames[Y],"        ", support, )
        i = i+1

In [11]:
freqItemsets = []
for kitemsets in F.values():
    for itemset in kitemsets:
        if len(itemset) > 0:
            freqItemsets.append(itemset)

associationRules = generateRules(dataset, freqItemsets, 0.8)
print("Association Rules at the frequency threshold 0.005 and the confidence threshold 0.8\n")
printAssociationRules(associationRules, itemNames)

Association Rules at the frequency threshold 0.005 and the confidence threshold 0.8

Support 		 Confidence 		 Association rule
1. 0.4023 		 1.0000 		 { Crew } => { Adult } 		 
2. 0.3918 		 0.9740 		 { Crew } => { Male } 		 
3. 0.1445 		 0.9815 		 { 1st class } => { Adult } 		 
4. 0.1186 		 0.9158 		 { 2nd class } => { Adult } 		 
5. 0.2850 		 0.8881 		 { 3rd class } => { Adult } 		 
6. 0.1932 		 0.9043 		 { Female } => { Adult } 		 
7. 0.7573 		 0.9630 		 { Male } => { Adult } 		 
8. 0.2968 		 0.9197 		 { Survived } => { Adult } 		 
9. 0.6536 		 0.9651 		 { Not survived } => { Adult } 		 
10. 0.6200 		 0.9154 		 { Not survived } => { Male } 		 
11. 0.0105 		 1.0000 		 { Crew, Female } => { Adult } 		 
12. 0.3918 		 0.9740 		 { Crew } => { Adult, Male } 		 
13. 0.3918 		 0.9740 		 { Crew, Adult } => { Male } 		 
14. 0.3918 		 1.0000 		 { Crew, Male } => { Adult } 		 
15. 0.0964 		 1.0000 		 { Crew, Survived } => { Adult } 		 
16. 0.3059 		 1.0000 		 { Crew, Not survived } => { Adult } 	

3. Now print only the association rules of the form X => {Survived} or X => {Not survived}

In [12]:
# We are interested only in the association rules of the form X => {Survived} or X => {Not survived}
associationRulesSurvived = []
for (X,Y,support,conf) in associationRules:
    if (Y == [8]) or (Y == [9]):
        associationRulesSurvived.append((X,Y,support,conf))

# Sort rules with respect to confidence
associationRulesSurvived = sorted(associationRulesSurvived, key=lambda x: x[3], reverse=True)
printAssociationRules(associationRulesSurvived, itemNames)

Support 		 Confidence 		 Association rule
1. 0.0109 		 1.0000 		 { 2nd class, Child } => { Survived } 		 
2. 0.0059 		 1.0000 		 { 2nd class, Child, Female } => { Survived } 		 
3. 0.0050 		 1.0000 		 { 2nd class, Child, Male } => { Survived } 		 
4. 0.0641 		 0.9724 		 { 1st class, Female } => { Survived } 		 
5. 0.0636 		 0.9722 		 { 1st class, Adult, Female } => { Survived } 		 
6. 0.0700 		 0.9167 		 { 2nd class, Adult, Male } => { Not survived } 		 
7. 0.0423 		 0.8774 		 { 2nd class, Female } => { Survived } 		 
8. 0.0091 		 0.8696 		 { Crew, Female } => { Survived } 		 
9. 0.0091 		 0.8696 		 { Crew, Adult, Female } => { Survived } 		 
10. 0.0700 		 0.8603 		 { 2nd class, Male } => { Not survived } 		 
11. 0.0364 		 0.8602 		 { 2nd class, Adult, Female } => { Survived } 		 
12. 0.1759 		 0.8377 		 { 3rd class, Adult, Male } => { Not survived } 		 
13. 0.1918 		 0.8275 		 { 3rd class, Male } => { Not survived } 		 
